# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)


### Upute za pokretanje

1) Pokrenuti donju liniju i viditi jel se sve moze importati
1) Skinuti sve sto se ne moze sa `!pip install` unutar notebooka
(pip bi trebao raditi i s condom ako je dobro namjesteno)
    - Skinuti pytorch sa stranice [Pytorch](https://pytorch.org/get-started/locally/) - odaberite CUDA 11.8
    ako imate graficku. Meni je bilo:
    `> !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118`
    - Ako nemate CUDA, notebook ce biti sporiji - dovoljan za inference, 
    ali nedovoljan za fine-tuning (sreca pa to niti ne radimo, bilo bi presporo 
    i s grafickom)
    - Skinuti huggingface transformers library:
    `> pip install 'transformers[torch]'`
1) Moze se sve importati na Kaggle, ovo mozda malo kasnije

In [81]:
import numpy as np
import json
from datasets import load_dataset

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

In [32]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    choice_to_index = {'A':0,'B':1,'C':2,'D':3, None:4}

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    for item in word_play.values():
        word_data_list.append(item)
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

In [205]:
def save(name=None, few_shot=False, example=format):
    try:
        os.mkdir('results')
    except:
        pass
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'results/{model_name}{inserted_name}_{i}.txt'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
                return

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load the model

In [82]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
model_path = 'microsoft/phi-2'

tokenizer = AutoTokenizer.from_pretrained(model_path)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [83]:
try:
    model = model.to(device)
except Exception as e:
    print(e)
    print("Changing device to cpu")
    device = 'cpu'

In [84]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [85]:
test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

Dataset length 903


In [86]:
format = """Question: {}
Choice:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:("""

In [87]:
fewshot_format = format + "{})"

In [88]:
def get_fewshot_demo(examples):
    label_to_answer = ['A','B','C','D']
    fewshot = ''
    for sample in examples:
        fewshot += fewshot_format.format(sample['Question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3],label_to_answer[int(sample['label'])])
    return fewshot

def get_single_demo(sample):
    sample_demo = format.format(
        sample['question'],
        sample['choice_list'][0],
        sample['choice_list'][1],
        sample['choice_list'][2],
        sample['choice_list'][3])
    return sample_demo

In [89]:
demonstration = get_fewshot_demo([{
        "Question": "Five kids were playing in a park when it suddenly started to rain. Four of the kids dashed towards a nearby tree for shelter, but the fifth did not. Yet, they all stayed dry, and all but the fifth were out of breath. How is that possible?",
        "Answer": "The fifth kid was in a picture that one of the other kids was holding.",
        "Choice1": "The 5th kid walked slowly  to dodge the snow.",
        "Choice2": "This kid walked much faster than they run",
        "Choice3": "None of above.",
        "label": 1,
        "choice_list": [
            "This kid walked much faster than they run",
            "The fifth kid was in a picture that one of the other kids was holding.",
            "The 5th kid walked slowly  to dodge the snow.",
            "None of above."
        ],
        "choice_order": [
            2,
            0,
            1,
            3
        ]
    }])
demonstration

'Question: Five kids were playing in a park when it suddenly started to rain. Four of the kids dashed towards a nearby tree for shelter, but the fifth did not. Yet, they all stayed dry, and all but the fifth were out of breath. How is that possible?\nChoice:\n(A) This kid walked much faster than they run\n(B) The fifth kid was in a picture that one of the other kids was holding.\n(C) The 5th kid walked slowly  to dodge the snow.\n(D) None of above.\n\nAnswer:(B)'

In [90]:
good_responses = [f'{x})' for x in ['A', 'B', 'C', 'D']]
good_responses

['A)', 'B)', 'C)', 'D)']

In [91]:
def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [157]:
def set_predictions(data_list):
    for index,item in enumerate(data_list):
        try:
            item['predict'] = None
            for x in ['A', 'B', 'C', 'D']:
                if (f'{x})') in item['response']:
                    item['predict'] = x

            if item['predict'] is None:
                print(index)
    #         item['predict'] = item['response'].split('(')[1].strip().split(')')[0].strip()
    #         if item['predict'] not in ['A', 'B', 'C', 'D']:
    #         raise Exception("Not valid response")
        except:
            item['predict'] = None
            print(index)

In [196]:
for i, sample in enumerate(generated_data_list):
    if i == 755: continue
    assert sample['distractor2'] not in sample['choice_list'], i
    assert sample['question'][:-1] in sample['choice_list'], i

In [183]:
from collections import defaultdict

def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    for sample in tqdm(samples):
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        original_tokens = len(inputs[0])
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample = False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        outputs = outputs[0][0 if all_tokens else original_tokens:]
        sample['response'] = tokenizer.decode(outputs)
    set_predictions(samples)

In [184]:
generate(test_data_list[:10], tokens=1000, all_tokens=False, few_shot=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [190]:
for sample in generated_data_list[:10]:
    print("\n>", sample['response'])


> C)

> B)

> B)

> B)

> B)

> A)

> B)

> C)

> B)

> B)


In [178]:
generated_data_list = []
for i in range(len(test_data_list)):
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    choice_list[current['choice_order'].index(2)] = current['question'][:-1]
    new['choice_list'] = choice_list
    generated_data_list.append(new)


In [181]:
len(generated_data_list)

903

In [186]:
generate(generated_data_list, tokens=20, all_tokens=False, few_shot=False)

  0%|          | 0/903 [00:00<?, ?it/s]

In [114]:
generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

  0%|          | 0/903 [00:00<?, ?it/s]

In [200]:
answered_w_question = 0
for sample in generated_data_list:
    predict = ['A', 'B', 'C', 'D'].index(sample['predict'])
    if sample['choice_order'][predict] == 2:
        answered_w_question += 1
answered_w_question, len(generated_data_list)

(527, 903)

In [197]:
word_play,sentence_play = getResultdata(generated_data_list)
# word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play, sentence_play)

#########Wordplay##########
over_all accuracy 0.24242424242424243
single_original_accuracy 0.25757575757575757
single_semantic_accuracy 0.22727272727272727
single_context_accuracy 0.24242424242424243
sr_accuracy 0.13636363636363635
cr_accuracy 0.06818181818181818
#########Sentence##########
over_all accuracy 0.17751479289940827
single_original_accuracy 0.15976331360946747
single_semantic_accuracy 0.1952662721893491
single_context_accuracy 0.17751479289940827
sr_accuracy 0.08875739644970414
cr_accuracy 0.029585798816568046
#########All data##########
over_all accuracy 0.2059800664451827
single_original_accuracy 0.2026578073089701
single_semantic_accuracy 0.20930232558139536
single_context_accuracy 0.2059800664451827
sr_accuracy 0.10963455149501661
cr_accuracy 0.046511627906976744


In [207]:
save(name="mixed_with_question", example=get_single_demo(generated_data_list[0]))

In [118]:
# save(few_shot=demonstration)
save()